In [ ]:
#|default_exp sync

# sync

> Propagate small changes in the library back to notebooks
- order: 6

The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbdev_update` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells or reorder cells with that functionality, so your corrections should remain limited.

In [ ]:
#|export
from nbdev.imports import *
from nbdev.config import *
from nbdev.maker import *
from nbdev.process import *
from nbdev.process import _partition_cell
from nbdev.export import *
from nbdev.doclinks import _iter_py_cells

from execnb.nbio import *
from fastcore.script import *
from fastcore.xtras import *

import ast
from importlib import import_module

In [ ]:
#|hide
from fastcore.test import *

In [ ]:
#|export
def absolute_import(name, fname, level):
    "Unwarps a relative import in `name` according to `fname`"
    if not level: return name
    mods = fname.split(os.path.sep)
    if not name: return '.'.join(mods)
    return '.'.join(mods[:len(mods)-level+1]) + f".{name}"

In [ ]:
test_eq(absolute_import('xyz', 'nbdev', 0), 'xyz')
test_eq(absolute_import('', 'nbdev', 1), 'nbdev')
test_eq(absolute_import(None, 'nbdev', 1), 'nbdev')
test_eq(absolute_import('core', 'nbdev', 1), 'nbdev.core')
test_eq(absolute_import('core', 'nbdev/vision', 2), 'nbdev.core')  # from ..core import *
test_eq(absolute_import('transform', 'nbdev/vision', 1), 'nbdev.vision.transform')  # from .transform import *
test_eq(absolute_import('notebook.core', 'nbdev/data', 2), 'nbdev.notebook.core')  # from ..notebook.core import *

In [ ]:
#|export
@functools.lru_cache(maxsize=None)
def _mod_files():
    midx = import_module(f'{get_config().lib_path.name}._modidx')
    return L(files for mod in midx.d['syms'].values() for _,files in mod.values()).unique()

In [ ]:
#|export
_re_import = re.compile("from\s+\S+\s+import\s+\S")

In [ ]:
#|hide
assert _re_import.match('from foo import bar')
assert not _re_import.match('#from foo import bar')

In [ ]:
#|export
def _to_absolute(code, py_path, lib_dir):
    if not _re_import.search(code): return code
    res = update_import(code, ast.parse(code).body, str(py_path.relative_to(lib_dir).parent), absolute_import)
    return ''.join(res) if res else code

In [ ]:
#|export
def _update_nb(nb_path, cells, lib_dir):
    "Update notebook `nb_path` with contents from `cells`"
    nbp = NBProcessor(nb_path, ExportModuleProc(), rm_directives=False)
    nbp.process()
    for cell in cells:
        assert cell.nb_path == nb_path
        nbcell = nbp.nb.cells[cell.idx]
        dirs,_ = _partition_cell(nbcell, 'python')
        nbcell.source = ''.join(dirs) + _to_absolute(cell.code, cell.py_path, lib_dir)
    write_nb(nbp.nb, nb_path)

In [ ]:
#|export
def _update_mod(py_path, lib_dir):
    "Propagate changes from cells in module `py_path` to corresponding notebooks"
    py_cells = L(_iter_py_cells(py_path)).filter(lambda o: o.nb != 'auto')
    for nb_path,cells in groupby(py_cells, 'nb_path').items(): _update_nb(nb_path, cells, lib_dir)

In [ ]:
#|hide
assert min(_mod_files().map(lambda x: x.endswith('.py'))) is True

ModuleNotFoundError: No module named '/home/birk/code/nbdev/nbdev'

In [ ]:
#|export
@call_parse
def nbdev_update(fname:str=None): # A Python file name to update
    "Propagate change in modules matching `fname` to notebooks that created them"
    if fname and fname.endswith('.ipynb'): raise ValueError("`nbdev_update` operates on .py files.  If you wish to convert notebooks instead, see `nbdev_export`.")
    if os.environ.get('IN_TEST',0): return
    cfg = get_config()
    fname = Path(fname or cfg.lib_path)
    lib_dir = cfg.lib_path.parent
    files = globtastic(fname, file_glob='*.py', skip_folder_re='^[_.]').filter(lambda x: str(Path(x).absolute().relative_to(lib_dir) in _mod_files()))
    files.map(_update_mod, lib_dir=lib_dir)

In [ ]:
#|hide
# nbdev_update("../nbdev/sync.py")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()